# Импорт модулей и констант (функции, и истинное значение интеграла)

In [2]:
import numpy as np
import plotly.graph_objects as go

from plotly.subplots import make_subplots

import pandas as pd
from abc import ABC, abstractmethod



In [3]:
TRUE_INTEGRAL= np.exp(1)* 2 + 6

In [4]:
def f_research(x):
    return np.sqrt(np.exp(x))+4*x

# Анализ методов подсчёта интегралов


In [5]:

class Integrator(ABC):
    @abstractmethod
    def integrate(self, a, b, n, f) :
        pass

class RectangleMethod(Integrator):
    def __init__(self, mode = "left"):
        self.mode = mode

    def integrate(self, a, b, n, f) :
        dx = (b - a) / n
        x0 = a + np.arange(n) * dx
    
        if self.mode == 'left':
            xi = x0
        elif self.mode == 'right':
            xi = x0 + dx
        elif self.mode == 'mid':
            xi = x0 + dx/2
        else:
            xi = np.random.uniform(x0, x0 + dx)
            
        return np.sum(f(xi)) * dx

class TrapezoidalMethod(Integrator):
    def integrate(self, a, b, n, f) :
        dx = (b - a) / n
        x = a + np.arange(1, n) * dx
        return (0.5*(f(a) + f(b)) + np.sum(f(x))) * dx

class SimpsonMethod(Integrator):
    def integrate(self, a, b, n, f) :
        
        dx=(b - a) / n
        x=a + np.arange(1, n) * dx
        coefs = np.where(np.arange(1, n) % 2 == 1, 4, 2)
        return (f(a) + f(b) + np.sum(coefs * f(x))) * dx / 3

In [6]:
a, b = 0.0, 2.0
methods = {
    "rect_l": RectangleMethod("left"),
    "rect_m": RectangleMethod("mid"),
    "rect_r": RectangleMethod("right"),
    "trapez": TrapezoidalMethod(),
    "simpson": SimpsonMethod()
}

results = []
for n in (2**k for k in range(8)):
    for name, method in methods.items():
        if name == "simpson" and n % 2 != 0:
            continue
        approx = method.integrate(a, b, n, f_research)
        results.append({
            "method": name, 
            "n": n, 
            "approx": approx, 
            "error": approx - TRUE_INTEGRAL
        })

df = pd.DataFrame(results)

In [7]:
metrics = df.groupby("method")['error'].agg(
    mse=lambda x: (x**2).mean(),
    mae=lambda x: x.abs().mean()
).reset_index()

print('True integral:', TRUE_INTEGRAL)
print(metrics)

True integral: 11.43656365691809
    method           mse       mae
0   rect_l  1.496771e+01  2.372973
1   rect_m  2.587848e-03  0.023321
2   rect_r  1.653467e+01  2.467187
3  simpson  1.925663e-07  0.000177
4   trapez  1.059871e-02  0.047107


In [8]:
methods = ['rect_l', 'rect_m', 'rect_r', 'trapez', 'simpson']
titles = {
    'rect_l': 'Метод левых прямоугольников',
    'rect_m': 'Метод средних прямоугольников',
    'rect_r': 'Метод правых прямоугольников',
    'trapez': 'Метод трапеций',
    'simpson': 'Метод Симпсона'
}

# Создаем список трассировок для каждого метода
traces = []
for method in methods:
    df_m = df[df['method'] == method]
    traces.append(
        go.Scatter(
            x=df_m['n'],
            y=df_m['error'].abs(),
            mode='lines+markers',
            name=titles[method],
            visible=(method == methods[0])
    )
    )

# Создаем кнопки для выбора методов
buttons = []
for i, method in enumerate(methods):
    visibility = [False] * len(methods)
    visibility[i] = True
    
    buttons.append(
        dict(
            label=titles[method],
            method='update',
            args=[{'visible': visibility},
                  {'title': {'text': titles[method]}}]
        )
    )

#
fig = go.Figure(data=traces)

fig.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            direction='down',
            x=0.95,  
            y=1.15,  
            xanchor='right',  
            showactive=True,
            buttons=buttons
        )
    ],
    title=dict(
        text=titles[methods[0]],
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='n (разбиений)',
        type='log',
        tickvals=df['n'].unique(),
        ticktext=df['n'].unique().astype(str),
        gridcolor='lightgray',
        exponentformat='none'
    ),
    yaxis=dict(
        title='Ошибка',
        gridcolor='lightgray'
    ),
    hovermode='x unified',
    plot_bgcolor='white',
    height=600,
    margin=dict(r=20)  
)

fig.update_layout(
    legend=dict(
        title='Методы',
        yanchor='top',
        y=0.99,
        xanchor='left',
        x=0.01
    )
)

fig.show()


# Визуализация

In [9]:


def plot_area_data(method, a, b, n, f):
    """
    Возвращает данные для построения графика функции и закрашенных областей
    
    Возвращает:
    (x_vals, y_vals, x_area, y_area) - данные основной функции и закрашенных областей
    """
    dx = (b - a) / n
    x_vals = np.linspace(a, b, 500)
    y_vals = f(x_vals)
    
    x_area = []
    y_area = []
    
    for i in range(n):
        x0 = a + i * dx
        x1 = x0 + dx
        
        if isinstance(method, RectangleMethod):
            if method.mode == "left":
                height = f(x0)
            elif method.mode == "right":
                height = f(x1)
            elif method.mode == "mid":
                height = f((x0 + x1) / 2)
            else:
                height = f(np.random.uniform(x0, x1))
            
            rect_x = [x0, x0, x1, x1]
            rect_y = [0, height, height, 0]
            
        elif isinstance(method, TrapezoidalMethod):
          
            y0 = f(x0)
            y1 = f(x1)
            rect_x = [x0, x0, x1, x1]
            rect_y = [0, y0, y1, 0 ]
            
        elif isinstance(method, SimpsonMethod):
   
            xi = np.linspace(x0, x1, 100)
            yi = f(xi) 
   
            rect_x = list(xi) + [x1, x0]
            rect_y = list(yi) + [0, 0]
 
        x_area.extend(rect_x)
        y_area.extend(rect_y)
        x_area.append(None)
        y_area.append(None)
    
    
    return (
        x_vals.tolist(), 
        y_vals.tolist(),
        x_area[:-1], 
        y_area[:-1]
    )

In [10]:
def create_integration_visualization(a, b, f, ns, methods, fig_width = 1600, fig_height = 500) :
    """
    Создает интерактивную визуализацию методов интегрирования с переключателем n
    
    Параметры:
    a, b - границы интегрирования
    f - интегрируемая функция
    ns - список значений n для анализа
    methods - словарь методов интегрирования {название: экземпляр метода}
    fig_width, fig_height - размеры фигуры
    
    Возвращает:
    plotly Figure объект с интерактивной визуализацией
    """
    fig = make_subplots(
        rows=1, 
        cols=len(methods),
        subplot_titles=list(methods.keys()),
        horizontal_spacing=0.07,
        vertical_spacing=0.2
    )

    traces_for_n = {n: [] for n in ns}

    for n in ns:
        for col, (title, method) in enumerate(methods.items(), 1):
            if isinstance(method, SimpsonMethod) and n % 2 != 0:
                # Блок с ошибкой 
                trace = go.Scatter(
                    x=[0.5], y=[0.5], 
                    mode="text",
                    text=["n должно быть чётным (("],
                    textfont=dict(size=14),
                    showlegend=False,
                    visible=(n == ns[0])
                )
                fig.add_trace(trace, row=1, col=col)
                traces_for_n[n].append(len(fig.data) - 1)
                
            else:
                # Блок с другими графиками
                x_vals, y_vals, x_area, y_area = plot_area_data(method, a, b, n, f)
                
                trace_func = go.Scatter(
                    x=x_vals, y=y_vals,
                    mode='lines',
                    line=dict(color='blue'),
                    showlegend=False,
                    visible=(n == ns[0]))
                
                trace_area = go.Scatter(
                    x=x_area, y=y_area,
                    fill='tozeroy',
                    mode='none',
                    fillcolor='rgba(255,0,0,0.3)',
                    showlegend=False,
                    visible=(n == ns[0]))
                
                fig.add_trace(trace_func, row=1, col=col)
                traces_for_n[n].append(len(fig.data) - 1)
                fig.add_trace(trace_area, row=1, col=col)
                traces_for_n[n].append(len(fig.data) - 1)

                fig.update_xaxes(
                    showgrid=True, 
                    gridwidth=1, 
                    gridcolor='lightgray',
                    row=1, 
                    col=col
                )
                fig.update_yaxes(
                    showgrid=True,
                    gridwidth=1, 
                    gridcolor='lightgray',
                    row=1, 
                    col=col
                )

    buttons = [
        dict(
            label=str(n),
            method='update',
            args=[{"visible": [i in traces_for_n[n] for i in range(len(fig.data))]}, 
                 {"title.text": f"Закрашенные области при n = {n}"}]
        ) for n in ns
    ]

    fig.update_layout(
        updatemenus=[dict(
            type="dropdown",
            direction="down",
            x=1.05, xanchor="left",
            y=1.1, yanchor="top",
            buttons=buttons,
            showactive=True
        )],
        title_text=f"Закрашенные области при n = {ns[0]}",
        title_x=0.5,
        margin=dict(t=100, b=50),
        width=fig_width,
        height=fig_height,
        font=dict(size=12))
    

    fig.update_traces(
        textposition="middle center",
        selector=dict(mode="text")
    )

    return fig


if __name__ == "__main__":
    a = 0
    b = 2
    f_research = f_research
    ns = [3, 4, 8, 16]
    methods_to_plot = {
        'Левые прям.': RectangleMethod('left'),
        'Средние прям.': RectangleMethod('mid'),
        'Правые прям.': RectangleMethod('right'),
        'Трапеции': TrapezoidalMethod(),
        'Симпсон': SimpsonMethod()
    }
    
    fig = create_integration_visualization(
        a=a, b=b, f=f_research,
        ns=ns, methods=methods_to_plot
    )
    fig.show()